In [1]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.distributions import Categorical
from collections import deque
import matplotlib.pyplot as plt

In [2]:
import gymnasium as gym
import ale_py
import AutoROM

In [3]:
import gymnasium as gym

env = gym.make("BeamRiderNoFrameskip-v4", render_mode="human")
obs, info = env.reset()
print("✅ Beam Rider environment loaded successfully!")
env.close()

✅ Beam Rider environment loaded successfully!


In [6]:
# for i in range(10):
#     obs, info = env.reset()
#     done = False

#     print(f"Episode {i+1}:")

#     while not done:
#         action = env.action_space.sample()
#         obs, reward, terminated, truncated, info = env.step(action)
#         done = terminated or truncated

#     break

In [4]:
class ActorCritic(nn.Module):
    def __init__(self, obs_shape, n_actions):
        super().__init__()
        c, h, w = obs_shape  # now c = 4
        self.conv = nn.Sequential(
            nn.Conv2d(c, 32, 8, stride=4), nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2), nn.ReLU(),
            nn.Conv2d(64, 64, 3, stride=1), nn.ReLU()
        )

        conv_out_size = self._get_conv_out(obs_shape)
        self.fc = nn.Sequential(nn.Linear(conv_out_size, 512), nn.ReLU())
        self.actor = nn.Linear(512, n_actions)
        self.critic = nn.Linear(512, 1)

    def _get_conv_out(self, shape):
        o = torch.zeros(1, *shape)
        o = self.conv(o)
        return int(np.prod(o.size()))

    def forward(self, x):
        x = x / 255.0  # normalize pixel values
        conv_out = self.conv(x).view(x.size(0), -1)
        fc_out = self.fc(conv_out)
        return self.actor(fc_out), self.critic(fc_out)

In [5]:
class PPOAgent:
    def __init__(self, obs_shape, n_actions, lr=2.5e-4, gamma=0.99, clip_eps=0.1, k_epochs=4):
        self.gamma = gamma
        self.clip_eps = clip_eps
        self.k_epochs = k_epochs

        self.policy = ActorCritic(obs_shape, n_actions)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr)

    def select_action(self, state):
        state = torch.FloatTensor(state).unsqueeze(0)  # (1, 4, 84, 84)
        with torch.no_grad():
            logits, value = self.policy(state)
        probs = torch.softmax(logits, dim=-1)
        dist = Categorical(probs)
        action = dist.sample()
        return action.item(), dist.log_prob(action), value.item()

    def compute_returns(self, rewards, dones, next_value):
        R = next_value
        returns = []
        for r, d in zip(reversed(rewards), reversed(dones)):
            R = r + self.gamma * R * (1 - d)
            returns.insert(0, R)
        return torch.tensor(returns, dtype=torch.float32)

    def update(self, memory):
        states = torch.FloatTensor(np.array(memory["states"]))
        actions = torch.LongTensor(memory["actions"])
        old_log_probs = torch.stack(memory["log_probs"]).detach()
        returns = torch.FloatTensor(memory["returns"])

        for _ in range(self.k_epochs):
            logits, values = self.policy(states)
            dist = Categorical(torch.softmax(logits, dim=-1))
            log_probs = dist.log_prob(actions)
            entropy = dist.entropy().mean()

            ratios = (log_probs - old_log_probs).exp()
            advantages = returns - values.squeeze().detach()

            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1 - self.clip_eps, 1 + self.clip_eps) * advantages

            actor_loss = -torch.min(surr1, surr2).mean()
            critic_loss = nn.MSELoss()(values.squeeze(), returns)
            loss = actor_loss + 0.5 * critic_loss - 0.01 * entropy

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()


In [6]:
def preprocess(obs):
    # Convert RGB (210x160x3) to grayscale + resize (84x84)
    import cv2
    obs_gray = cv2.cvtColor(obs, cv2.COLOR_RGB2GRAY)
    obs_resized = cv2.resize(obs_gray, (84, 84), interpolation=cv2.INTER_AREA)
    return obs_resized

In [7]:
def train_beamrider(episodes=500, log_interval=10, save_path="beamrider_ppo.pth"):
    import os
    from datetime import datetime

    run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_dir = os.path.join("runs", f"beamrider_{run_id}")
    os.makedirs(log_dir, exist_ok=True)
    log_file_path = os.path.join(log_dir, "training_log.csv")

    # Initialize CSV log file
    if not os.path.exists(log_file_path):
        with open(log_file_path, "w") as f:
            f.write("episode,total_reward,avg_reward_10,best_reward\n")

    # Use the working legacy name (not ALE/)
    env = gym.make("BeamRiderNoFrameskip-v4", render_mode="human")
    obs, _ = env.reset()
    obs = preprocess(obs)
    state_shape = (4, 84, 84)  # 4 stacked frames
    n_actions = env.action_space.n

    agent = PPOAgent(state_shape, n_actions)
    reward_history = []
    best_avg_reward = -float("inf")

    frame_stack = deque(maxlen=4)
    for _ in range(4):
        frame_stack.append(np.zeros((84, 84), dtype=np.uint8))

    for ep in range(episodes):
        obs, _ = env.reset()
        frame_stack.extend([preprocess(obs)] * 4)
        total_reward = 0
        done = False

        memory = {"states": [], "actions": [], "log_probs": [], "rewards": [], "dones": [], "returns": []}

        while not done:
            state = np.stack(frame_stack, axis=0)
            action, log_prob, value = agent.select_action(state)
            obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            frame_stack.append(preprocess(obs))

            memory["states"].append(state)
            memory["actions"].append(action)
            memory["log_probs"].append(log_prob)
            memory["rewards"].append(reward)
            memory["dones"].append(done)
            total_reward += reward

        with torch.no_grad():
            state = np.stack(frame_stack, axis=0)
            _, next_value = agent.policy(torch.FloatTensor(state).unsqueeze(0))
            next_value = next_value.item()

        memory["returns"] = agent.compute_returns(memory["rewards"], memory["dones"], next_value)
        agent.update(memory)

        reward_history.append(total_reward)

        # Compute recent average reward
        recent_rewards = reward_history[-10:] if len(reward_history) >= 10 else reward_history
        avg_r_10 = float(np.mean(recent_rewards))

        # Track best model based on 10-episode average
        is_best = avg_r_10 > best_avg_reward
        if is_best:
            best_avg_reward = avg_r_10
            torch.save({
                "policy_state_dict": agent.policy.state_dict(),
                "optimizer_state_dict": agent.optimizer.state_dict(),
                "reward_history": reward_history,
                "episode": ep + 1,
                "best_avg_reward": best_avg_reward
            }, os.path.join(log_dir, "best_model.pth"))

        # Append to CSV log
        with open(log_file_path, "a") as f:
            f.write(f"{ep+1},{total_reward},{avg_r_10},{best_avg_reward}\n")

        # Console logging
        print(f"Episode {ep+1}: Reward = {total_reward:.1f}, Avg10 = {avg_r_10:.2f}, BestAvg10 = {best_avg_reward:.2f}")

        # Periodic checkpoint saving
        if (ep + 1) % log_interval == 0:
            ckpt_path = os.path.join(log_dir, f"checkpoint_ep{ep+1}.pth")
            torch.save({
                "policy_state_dict": agent.policy.state_dict(),
                "optimizer_state_dict": agent.optimizer.state_dict(),
                "reward_history": reward_history,
                "episode": ep + 1,
                "best_avg_reward": best_avg_reward
            }, ckpt_path)
            print(f"Saved checkpoint to {ckpt_path}")

    env.close()
    plt.plot(reward_history)
    plt.title("PPO on BeamRider-v4")
    plt.xlabel("Episode")
    plt.ylabel("Reward")
    plt.show()

In [11]:
train_beamrider(episodes=200)

Episode 1: Reward = 308.0, Avg10 = 308.00, BestAvg10 = 308.00
Episode 2: Reward = 176.0, Avg10 = 242.00, BestAvg10 = 308.00
Episode 2: Reward = 176.0, Avg10 = 242.00, BestAvg10 = 308.00
Episode 3: Reward = 220.0, Avg10 = 234.67, BestAvg10 = 308.00
Episode 3: Reward = 220.0, Avg10 = 234.67, BestAvg10 = 308.00
Episode 4: Reward = 308.0, Avg10 = 253.00, BestAvg10 = 308.00
Episode 4: Reward = 308.0, Avg10 = 253.00, BestAvg10 = 308.00
Episode 5: Reward = 264.0, Avg10 = 255.20, BestAvg10 = 308.00
Episode 5: Reward = 264.0, Avg10 = 255.20, BestAvg10 = 308.00
Episode 6: Reward = 308.0, Avg10 = 264.00, BestAvg10 = 308.00
Episode 6: Reward = 308.0, Avg10 = 264.00, BestAvg10 = 308.00
Episode 7: Reward = 132.0, Avg10 = 245.14, BestAvg10 = 308.00
Episode 7: Reward = 132.0, Avg10 = 245.14, BestAvg10 = 308.00
Episode 8: Reward = 440.0, Avg10 = 269.50, BestAvg10 = 308.00
Episode 8: Reward = 440.0, Avg10 = 269.50, BestAvg10 = 308.00
Episode 9: Reward = 484.0, Avg10 = 293.33, BestAvg10 = 308.00
Episode 

KeyboardInterrupt: 

In [ ]:
import os
import glob
import torch
import numpy as np

# Utility to find the most recent run directory in runs/
def get_latest_run_dir(base_dir="runs"):
    if not os.path.exists(base_dir):
        return None
    subdirs = [os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
    if not subdirs:
        return None
    # Sort by modified time (latest last)
    subdirs.sort(key=os.path.getmtime)
    return subdirs[-1]

def continue_training_from_last_run(extra_episodes=50, checkpoint_name=None, log_interval=10):
    latest_run = get_latest_run_dir("runs")
    if latest_run is None:
        print("No previous runs found in 'runs/'. Start a new training run first.")
        return

    if checkpoint_name is None:
        checkpoint_path = os.path.join(latest_run, "checkpoint_ep90.pth")
    else:
        checkpoint_path = os.path.join(latest_run, checkpoint_name)

    if not os.path.exists(checkpoint_path):
        print(f"Checkpoint {checkpoint_path} not found.\nAvailable files: {os.listdir(latest_run)}")
        return

    print(f"Loading checkpoint from: {checkpoint_path}")
    ckpt = torch.load(checkpoint_path, map_location="cpu")

    # Recreate environment and agent
    env = gym.make("BeamRiderNoFrameskip-v4", render_mode="human")
    obs, _ = env.reset()
    obs = preprocess(obs)
    state_shape = (4, 84, 84)
    n_actions = env.action_space.n

    agent = PPOAgent(state_shape, n_actions)
    agent.policy.load_state_dict(ckpt["policy_state_dict"])
    agent.optimizer.load_state_dict(ckpt["optimizer_state_dict"])

    # Restore previous reward history and episode index if available
    reward_history = ckpt.get("reward_history", [])
    start_episode = ckpt.get("episode", len(reward_history))
    best_avg_reward = ckpt.get("best_avg_reward", -float("inf"))

    log_file_path = os.path.join(latest_run, "training_log.csv")
    if not os.path.exists(log_file_path):
        # If for some reason the CSV is missing, re-create with header
        with open(log_file_path, "w") as f:
            f.write("episode,total_reward,avg_reward_10,best_reward\n")

    frame_stack = deque(maxlen=4)
    for _ in range(4):
        frame_stack.append(np.zeros((84, 84), dtype=np.uint8))

    for ep_offset in range(extra_episodes):
        ep = start_episode + ep_offset
        obs, _ = env.reset()
        frame_stack.extend([preprocess(obs)] * 4)
        total_reward = 0
        done = False

        memory = {"states": [], "actions": [], "log_probs": [], "rewards": [], "dones": [], "returns": []}

        while not done:
            state = np.stack(frame_stack, axis=0)
            action, log_prob, value = agent.select_action(state)
            obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            frame_stack.append(preprocess(obs))

            memory["states"].append(state)
            memory["actions"].append(action)
            memory["log_probs"].append(log_prob)
            memory["rewards"].append(reward)
            memory["dones"].append(done)
            total_reward += reward

        with torch.no_grad():
            state = np.stack(frame_stack, axis=0)
            _, next_value = agent.policy(torch.FloatTensor(state).unsqueeze(0))
            next_value = next_value.item()

        memory["returns"] = agent.compute_returns(memory["rewards"], memory["dones"], next_value)
        agent.update(memory)

        reward_history.append(total_reward)
        recent_rewards = reward_history[-10:] if len(reward_history) >= 10 else reward_history
        avg_r_10 = float(np.mean(recent_rewards))

        # Update best model if improved
        is_best = avg_r_10 > best_avg_reward
        if is_best:
            best_avg_reward = avg_r_10
            torch.save({
                "policy_state_dict": agent.policy.state_dict(),
                "optimizer_state_dict": agent.optimizer.state_dict(),
                "reward_history": reward_history,
                "episode": ep + 1,
                "best_avg_reward": best_avg_reward
            }, os.path.join(latest_run, "best_model.pth"))

        # Append to CSV log
        with open(log_file_path, "a") as f:
            f.write(f"{ep+1},{total_reward},{avg_r_10},{best_avg_reward}\n")

        print(f"[CONT] Episode {ep+1}: Reward = {total_reward:.1f}, Avg10 = {avg_r_10:.2f}, BestAvg10 = {best_avg_reward:.2f}")

        # Periodic checkpoint during continued training
        if (ep + 1) % log_interval == 0:
            ckpt_path = os.path.join(latest_run, f"checkpoint_ep{ep+1}.pth")
            torch.save({
                "policy_state_dict": agent.policy.state_dict(),
                "optimizer_state_dict": agent.optimizer.state_dict(),
                "reward_history": reward_history,
                "episode": ep + 1,
                "best_avg_reward": best_avg_reward
            }, ckpt_path)
            print(f"Saved continued-training checkpoint to {ckpt_path}")

    env.close()
    print(f"Finished continuing training for {extra_episodes} episodes in run folder: {latest_run}")

In [11]:
continue_training_from_last_run(extra_episodes=10, log_interval=10)

Loading checkpoint from: runs\beamrider_20251119_140617\checkpoint_ep90.pth
[CONT] Episode 91: Reward = 264.0, Avg10 = 382.80, BestAvg10 = 542.00
[CONT] Episode 92: Reward = 396.0, Avg10 = 360.80, BestAvg10 = 542.00
[CONT] Episode 93: Reward = 264.0, Avg10 = 360.80, BestAvg10 = 542.00
[CONT] Episode 94: Reward = 220.0, Avg10 = 365.20, BestAvg10 = 542.00
[CONT] Episode 95: Reward = 176.0, Avg10 = 343.20, BestAvg10 = 542.00
[CONT] Episode 96: Reward = 572.0, Avg10 = 338.80, BestAvg10 = 542.00
[CONT] Episode 97: Reward = 396.0, Avg10 = 360.80, BestAvg10 = 542.00
[CONT] Episode 98: Reward = 220.0, Avg10 = 316.80, BestAvg10 = 542.00
[CONT] Episode 99: Reward = 220.0, Avg10 = 308.00, BestAvg10 = 542.00
[CONT] Episode 100: Reward = 220.0, Avg10 = 294.80, BestAvg10 = 542.00
Saved continued-training checkpoint to runs\beamrider_20251119_140617\checkpoint_ep100.pth
Finished continuing training for 10 episodes in run folder: runs\beamrider_20251119_140617
